In [1]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets
from pathlib import Path

sns.set(font_scale=2)
sns.set_theme(style="whitegrid")

data_dir = Path('/Users/suzinyou/Dropbox/가천대 서울연구원 프로젝트/10. 데이터산출물')

# Preprocess (지표를 평균에 병합)

In [1]:
mm = pd.read_excel(data_dir / '02_mean_median' / 'mean_median.xlsx', sheet_name=None)
indices = pd.read_excel(data_dir / '04_indices' / 'indices.xlsx', sheet_name=None)

In [2]:
list(mm.keys())

['KR_adult20',
 'KR_adult20_earner',
 'KR_capita',
 'KR_earner',
 'KR_adult15',
 'KR_sido_adult20',
 'KR_hh',
 'SEOUL_hh',
 'KR_eq',
 'SEOUL_eq',
 'SEOUL_sigungu_hh',
 'SEOUL_adult20',
 'SEOUL_adult15',
 'SEOUL_adult20_earner',
 'SEOUL_capita',
 'SEOUL_earner',
 'SEOUL_sigungu_adult20',
 'SEOUL_sigungu_adult20_earner']

In [3]:
list(indices.keys())

['seoul_adult20',
 'kr_adult20',
 'kr_adult20_earner',
 'seoul_sigungu_adult20',
 'seoul_adult20_earner',
 'seoul_sigungu_adult20_earner',
 'kr_adult15',
 'seoul_adult15']

In [4]:
indices['seoul_sigungu_adult20'].head()

,var,STD_YYYY,sigungu,gini,iqsr,rpr
0,inc_tot,2006,110,0.859586,NaN,0.553010
1,inc_tot,2006,140,0.841777,NaN,0.556682
2,inc_tot,2006,170,0.854507,NaN,0.534403
3,inc_tot,2006,200,0.797709,NaN,0.530484
4,inc_tot,2006,215,0.790238,NaN,0.514276


In [10]:
mm['SEOUL_sigungu_adult20'].head().merge(indices['seoul_sigungu_adult20'], on=['STD_YYYY', 'var', 'sigungu'])

,var,STD_YYYY,sigungu,count,mean,median,num_indi,frac_earners,gini,iqsr,rpr
0,inc_tot,2006,110,139983,1.460015e+07,0.0,181861,0.769725,0.859586,NaN,0.553010
1,inc_tot,2006,140,112655,1.174040e+07,0.0,157162,0.716808,0.841777,NaN,0.556682
2,inc_tot,2006,170,189619,1.567952e+07,0.0,238901,0.793714,0.854507,NaN,0.534403
3,inc_tot,2006,200,269801,1.057160e+07,0.0,344264,0.783704,0.797709,NaN,0.530484
4,inc_tot,2006,215,295602,1.119759e+07,0.0,380913,0.776035,0.790238,NaN,0.514276


In [9]:
mm_sheet_names = {k.lower():k for k in mm.keys()}
for k, df in indices.items():
    if k not in mm_sheet_names:
        continue
    mmk = mm_sheet_names[k]
    on_cols = ['STD_YYYY', 'var']
    if 'sigungu' in k:
        on_cols.append('sigungu')
    mm[mmk] = mm[mmk].merge(indices[k], on=on_cols)

In [10]:
with pd.ExcelWriter(data_dir / 'mean_median_and_indices.xlsx') as writer:
    for k, df in mm.items():
        df.to_excel(writer, index=False, sheet_name=k.lower())

# 실질소득 계산, 지역 이름 붙이기 등

In [2]:
mm = pd.read_excel(data_dir / 'mean_median_and_indices.xlsx', sheet_name=None)

In [10]:
map_dir = data_dir / 'external' / 'maps'

In [52]:
def load_cpi(region):
    """Load consumer price index data"""
    cpi = pd.read_excel(data_dir / 'external' / '소비자물가지수_서울과 전국__20201113113326.xlsx',
                        index_col='시도별(2)').T.iloc[2:]
    cpi /= 100
    cpi = cpi.reset_index().rename(
        columns={'index': 'std_yyyy', '소계': '전국', '서울특별시': '서울'}
    ).astype(
        {'std_yyyy': int, '전국': float, '서울': float}
    )
    if region == '서울':
        return cpi.drop(['전국'], axis=1).rename(columns={'서울': 'cpi'})
    if region == '전국':
        return cpi.drop(['서울'], axis=1).rename(columns={'전국': 'cpi'})
    return cpi


def load_sido_names():
    sidodf = pd.read_csv(map_dir / 'CTPRVN.csv')
    return sidodf[['CTPRVN_CD', 'CTPRVN_KOR_NM']]


def load_sigungu_names():
    sigungudf = pd.read_csv(map_dir / 'SIG.csv')
    seoul_mask = sigungudf['SIG_CD'].apply(lambda x: str(x).startswith('11'))
    sigungudf = sigungudf.loc[seoul_mask, ['SIG_CD', 'SIG_KOR_NM']]
    sigungudf['SIG_CD'] = sigungudf['SIG_CD'].apply(lambda x: x % 1000)
    sigungudf = sigungudf.rename(columns={'SIG_CD': 'sigungu', 'SIG_KOR_NM': '구'})
    return sigungudf

In [53]:
cpi = load_cpi(region='서울')

In [57]:
cpi.head()

시도별(2),std_yyyy,cpi
0,1975,0.09953
1,1976,0.11490
2,1977,0.12668
3,1978,0.14485
4,1979,0.17104


In [44]:
sigungu = load_sigungu_names()

In [45]:
sigungu

,sigungu,구
0,110,종로구
1,140,중구
2,170,용산구
3,200,성동구
4,215,광진구
5,230,동대문구
6,260,중랑구
7,290,성북구
8,305,강북구
9,320,도봉구


In [16]:
sido  = load_sido_names()

In [59]:
df = mm['seoul_sigungu_adult20']
if 'STD_YYYY' in df.columns:
    df = df.rename(columns={'STD_YYYY': 'std_yyyy'})

In [62]:
df = df.merge(sigungu, on='sigungu', how='left', ).merge(cpi, on='std_yyyy', how='left')

In [63]:
df['mean_real'] = df['mean'] / df.cpi
df['median_real'] = df['median'] / df.cpi

In [64]:
df

,var,std_yyyy,sigungu,count,mean,median,num_indi,frac_earners,gini,iqsr,rpr,구,cpi,mean_real,median_real
0,inc_tot,2006,110,139983,1.460015e+07,0.0,181861,0.769725,0.859586,NaN,0.553010,종로구,0.79685,1.832233e+07,0.0
1,inc_tot,2006,140,112655,1.174040e+07,0.0,157162,0.716808,0.841777,NaN,0.556682,중구,0.79685,1.473351e+07,0.0
2,inc_tot,2006,170,189619,1.567952e+07,0.0,238901,0.793714,0.854507,NaN,0.534403,용산구,0.79685,1.967688e+07,0.0
3,inc_tot,2006,200,269801,1.057160e+07,0.0,344264,0.783704,0.797709,NaN,0.530484,성동구,0.79685,1.326674e+07,0.0
4,inc_tot,2006,215,295602,1.119759e+07,0.0,380913,0.776035,0.790238,NaN,0.514276,광진구,0.79685,1.405231e+07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,prop_txbs_tot,2018,620,457717,5.946420e+07,0.0,520625,0.879168,0.855651,NaN,0.683743,관악구,1.04580,5.686001e+07,0.0
646,prop_txbs_tot,2018,650,357750,2.620108e+08,0.0,443171,0.807250,0.818521,NaN,0.536031,서초구,1.04580,2.505362e+08,0.0
647,prop_txbs_tot,2018,680,456384,2.793685e+08,0.0,556060,0.820746,0.837791,NaN,0.561159,강남구,1.04580,2.671337e+08,0.0
648,prop_txbs_tot,2018,710,551424,1.451885e+08,0.0,669718,0.823367,0.825749,NaN,0.596670,송파구,1.04580,1.388301e+08,0.0


In [65]:
df.columns.tolist()

['var',
 'std_yyyy',
 'sigungu',
 'count',
 'mean',
 'median',
 'num_indi',
 'frac_earners',
 'gini',
 'iqsr',
 'rpr',
 '구',
 'cpi',
 'mean_real',
 'median_real']

In [66]:
df[['var',
    'std_yyyy',
    '구',
    'count',
    'num_indi',
    'frac_earners',
    'mean',
    'mean_real',
    'median',
    'median_real',
    'gini',
    'iqsr',
    'rpr',]]

,var,std_yyyy,구,count,num_indi,frac_earners,mean,mean_real,median,median_real,gini,iqsr,rpr
0,inc_tot,2006,종로구,139983,181861,0.769725,1.460015e+07,1.832233e+07,0.0,0.0,0.859586,NaN,0.553010
1,inc_tot,2006,중구,112655,157162,0.716808,1.174040e+07,1.473351e+07,0.0,0.0,0.841777,NaN,0.556682
2,inc_tot,2006,용산구,189619,238901,0.793714,1.567952e+07,1.967688e+07,0.0,0.0,0.854507,NaN,0.534403
3,inc_tot,2006,성동구,269801,344264,0.783704,1.057160e+07,1.326674e+07,0.0,0.0,0.797709,NaN,0.530484
4,inc_tot,2006,광진구,295602,380913,0.776035,1.119759e+07,1.405231e+07,0.0,0.0,0.790238,NaN,0.514276
...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,prop_txbs_tot,2018,관악구,457717,520625,0.879168,5.946420e+07,5.686001e+07,0.0,0.0,0.855651,NaN,0.683743
646,prop_txbs_tot,2018,서초구,357750,443171,0.807250,2.620108e+08,2.505362e+08,0.0,0.0,0.818521,NaN,0.536031
647,prop_txbs_tot,2018,강남구,456384,556060,0.820746,2.793685e+08,2.671337e+08,0.0,0.0,0.837791,NaN,0.561159
648,prop_txbs_tot,2018,송파구,551424,669718,0.823367,1.451885e+08,1.388301e+08,0.0,0.0,0.825749,NaN,0.596670


In [67]:
regcd = pd.read_csv(data_dir / 'external'/ '(양식)customized_DB_application_200707' / '코드설명_동읍면-Table 1.csv')

In [68]:
regcd

,SIDO_CD,SIDO_SGG_CD,SIDO_SGG_EMD_CD,ADDR_CD,ZN_TYPE_CD,SIDO_NM,SGG_NM,EMD_NM
0,11,11110,11110510,1111051000,3,서울특별시,종로구,청운동
1,11,11110,11110515,1111051500,3,서울특별시,종로구,청운효자동
2,11,11110,11110520,1111052000,3,서울특별시,종로구,효자동
3,11,11110,11110530,1111053000,3,서울특별시,종로구,사직동
4,11,11110,11110540,1111054000,3,서울특별시,종로구,삼청동
...,...,...,...,...,...,...,...,...
27220,50,50130,50130600,5013060000,4,제주특별자치도,서귀포시,대천동
27221,50,50130,50130610,5013061000,4,제주특별자치도,서귀포시,중문동
27222,50,50130,50130620,5013062000,4,제주특별자치도,서귀포시,예래동
27223,90,90900,90900901,9090090100,3,NaN,NaN,개성공업지구


In [73]:
regcd[['SIDO_SGG_CD', 'SGG_NM']].groupby('SIDO_SGG_CD').first().reset_index()

,SIDO_SGG_CD,SGG_NM
0,11110,종로구
1,11140,중구
2,11170,용산구
3,11200,성동구
4,11215,광진구
...,...,...
285,49720,남제주군
286,50110,제주시
287,50130,서귀포시
288,90900,NaN


In [78]:
regcd = pd.read_csv(data_dir / 'external' / '(양식)customized_DB_application_200707' / '코드설명_동읍면-Table 1.csv')
seoul_mask = regcd['SIDO_SGG_CD'].apply(lambda x: str(x).startswith('11'))
sigungudf = regcd.loc[seoul_mask, ['SIDO_SGG_CD', 'SGG_NM']].groupby('SIDO_SGG_CD').first().reset_index()
sigungudf['SIDO_SGG_CD'] = sigungudf['SIDO_SGG_CD'].apply(lambda x: int(x) % 1000)
sigungudf = sigungudf.rename(columns={'SIDO_SGG_CD': 'sigungu', 'SGG_NM': '구'})

In [80]:
sigungudf.dtypes

sigungu     int64
구          object
dtype: object

In [88]:
regcd = pd.read_csv(data_dir / 'external' / '(양식)customized_DB_application_200707' / '코드설명_동읍면-Table 1.csv')
regcd = regcd[['SIDO_CD', 'SIDO_NM']].groupby('SIDO_CD').first().reset_index().iloc[:-1]
regcd = regcd.rename(columns={'SIDO_CD': 'sido', 'SIDO_NM': '시도'})
regcd.sido = regcd.sido.astype(int)

In [87]:
regcd.dtypes

sido    object
시도      object
dtype: object